# Setup

In [90]:
import os
from pathlib import Path
import random

from cmdstanpy import cmdstan_path, CmdStanModel
import numpy as np
import pandas as pd
import cmdstanpy
import arviz
import seaborn as sns
import arviz as az

In [15]:
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
pandas2ri.activate()
readRDS = robjects.r['readRDS']

Using libraries at paths:
- /home/nels/R/x86_64-pc-linux-gnu-library/4.1
- /usr/local/lib/R/site-library
- /usr/lib/R/site-library
- /usr/lib/R/library


In [8]:
random.seed(42)
rng = np.random.default_rng(42)

# Course data

In [17]:
pest_data =  readRDS('../data/pest_data.RDS')
standata_hier = readRDS('../data/standata_hier.RDS')

In [83]:
pest_data

,building_id,date,traps,floors,sq_footage_p_floor,live_in_super,monthly_average_rent,average_tenant_age,age_of_building,total_sq_foot,month,complaints,log_sq_foot_1e4
1,37,17181.0,8.0,8.0,5149.008112,0.0,3846.949050,53.877424,47.0,41192.064892,1.0,1.0,1.415661
2,37,17211.0,8.0,8.0,5149.008112,0.0,3846.949050,53.877424,47.0,41192.064892,2.0,3.0,1.415661
3,37,17241.0,9.0,8.0,5149.008112,0.0,3846.949050,53.877424,47.0,41192.064892,3.0,0.0,1.415661
4,37,17271.0,10.0,8.0,5149.008112,0.0,3846.949050,53.877424,47.0,41192.064892,4.0,1.0,1.415661
5,37,17301.0,11.0,8.0,5149.008112,0.0,3846.949050,53.877424,47.0,41192.064892,5.0,0.0,1.415661
...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,98,17391.0,3.0,13.0,4557.786883,1.0,3785.183548,42.138240,39.0,59251.229483,8.0,6.0,1.779201
117,98,17421.0,2.0,13.0,4557.786883,1.0,3785.183548,42.138240,39.0,59251.229483,9.0,16.0,1.779201
118,98,17451.0,2.0,13.0,4557.786883,1.0,3785.183548,42.138240,39.0,59251.229483,10.0,5.0,1.779201
119,98,17481.0,2.0,13.0,4557.786883,1.0,3785.183548,42.138240,39.0,59251.229483,11.0,5.0,1.779201


In [84]:
pest_data.dtypes

building_id               int32
date                    float64
traps                   float64
floors                  float64
sq_footage_p_floor      float64
live_in_super           float64
monthly_average_rent    float64
average_tenant_age      float64
age_of_building         float64
total_sq_foot           float64
month                   float64
complaints              float64
log_sq_foot_1e4         float64
dtype: object

# Prior Predictive Checks

In [12]:
def simple_poisson_dgp(traps, alpha_mean, alpha_sd, beta_mean, beta_sd):
    n = len(traps)
    alpha = rng.normal(loc=alpha_mean, scale=alpha_sd, size=1)
    beta = rng.normal(loc=beta_mean, scale=beta_sd, size=1)
    complaints = rng.poisson(lam=np.exp(alpha + beta * traps), size=n)
    return complaints

## Sample from the priors

In [57]:
simple_poisson_dgp(
    traps=pest_data['traps'],
    alpha_mean=0,
    alpha_sd=1,
    beta_mean=0,
    beta_sd=1
)

array([  1915228,   1913960,  12173311,  77388556, 492030656, 492007184,
        77394564,  77393440,  12172754,  12174048,   1916015,  12174766,
          301619,    301479,   1915170,  12173968,  77386513,  12171783,
        12172046,  77402433,  77397211,  12172875,  12170326,   1916062,
         1916007,   1912988,    300888,     47253,     47204,    301127,
           47589,    300655,    300630,    301435,    301889,   1913804,
           47586,     47615,    301603,    301711,     47333,    301865,
          299624,     47549,    302054,     47267,    301797,   1915317,
          300887,    300970,   1916293,    301276,     47233,     47192,
          300579,    301402,     47120,     47429,     46946,    301919,
          300420,    300767,    301169,   1916437,    300273,    301188,
          300433,     47079,     47562,      7356,     47431,    301033,
            7377,      7432,     47283,     47215,    301446,   1913976,
        12170756,  12175378,  12174512,  12169340, 

In [47]:
# take 1000 samples
prior_preds = pd.DataFrame(
    [
        simple_poisson_dgp(
            traps=pest_data['traps'],
            alpha_mean=0,
            alpha_sd=1,
            beta_mean=0,
            beta_sd=1
        ) for _ in range(1000)
    ]
)

In [52]:
prior_preds.shape

(1000, 120)

In [62]:
prior_preds.transpose().mean().describe()

count    1.000000e+03
mean     9.473313e+10
std      2.970072e+12
min      0.000000e+00
25%      4.166667e-02
50%      1.012500e+00
75%      2.427417e+02
max      9.392069e+13
dtype: float64

In [63]:
prior_preds.transpose().min().describe()

count    1000.000000
mean        1.727000
std         4.810365
min         0.000000
25%         0.000000
50%         0.000000
75%         1.000000
max        79.000000
dtype: float64

In [64]:
prior_preds.transpose().max().describe()

count    1.000000e+03
mean     4.894503e+12
std      1.535367e+14
min      0.000000e+00
25%      1.000000e+00
50%      5.000000e+00
75%      1.913250e+03
max      4.855203e+15
dtype: float64

# First Stan model

In [85]:
standata_simple = {
    'N': 120, 
    'traps': pest_data.traps.astype(int).values, 
    'complaints': pest_data.complaints.astype(int).values
}

In [86]:
comp_simple_pois = CmdStanModel(stan_file='../stan_programs/simple_poisson_regression.stan')

INFO:cmdstanpy:found newer exe file, not recompiling
INFO:cmdstanpy:compiled model file: /home/nels/Workbench/bayestan/lander-stan-class-2021/stan_programs/simple_poisson_regression


In [87]:
fit_simple_pois = comp_simple_pois.sample(
    data=standata_simple,
    chains=4,
    iter_warmup=1000,
    iter_sampling=2000
)

INFO:cmdstanpy:start chain 1
INFO:cmdstanpy:start chain 2
INFO:cmdstanpy:start chain 3
INFO:cmdstanpy:start chain 4
INFO:cmdstanpy:finish chain 1
INFO:cmdstanpy:finish chain 2
INFO:cmdstanpy:finish chain 3
INFO:cmdstanpy:finish chain 4


In [88]:
fit_simple_pois.summary()

,Mean,MCSE,StdDev,5%,50%,95%,N_Eff,N_Eff/s,R_hat
name,,,,,,,,,
lp__,160.00,0.02300,1.000,160.00,160.00,160.00,1800.0,2800.0,1.0
alpha,2.60,0.00400,0.150,2.30,2.60,2.80,1500.0,2200.0,1.0
beta,-0.19,0.00062,0.023,-0.23,-0.19,-0.16,1400.0,2100.0,1.0
y_rep[1],2.80,0.01900,1.700,0.00,3.00,6.00,7815.0,11806.0,1.0
y_rep[2],2.90,0.02000,1.700,0.00,3.00,6.00,7634.0,11531.0,1.0
...,...,...,...,...,...,...,...,...,...
y_rep[116],7.50,0.03300,2.800,3.00,7.00,12.00,7050.0,10650.0,1.0
y_rep[117],9.10,0.03900,3.200,4.00,9.00,15.00,6709.0,10134.0,1.0
y_rep[118],9.20,0.04200,3.200,4.00,9.00,15.00,5919.0,8941.0,1.0


In [98]:
az_data = az.from_cmdstanpy(
    posterior=fit_simple_pois,
#    posterior_predictive="y_rep",
    observed_data={"traps": pest_data.traps},
#    log_likelihood="complaints",
#     coords={"school": np.arange(eight_school_data["J"])},
#     dims={
#         "theta": ["school"],
#         "y": ["school"],
#         "log_lik": ["school"],
#         "y_hat": ["school"],
#         "theta_tilde": ["school"],
#     },
)

In [99]:
az_data

Inference data with groups:
	> posterior
	> sample_stats
	> observed_data

In [100]:
az.plot_ppc(az_data)

TypeError: `data` argument must have the group "posterior_predictive" for ppcplot